In [2]:
from dtu_ctfds_02807_proj.context_creation import create_context

local = True
sc, spark = create_context(local)

Already been executed once, not running again!


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType, LongType, DoubleType, ArrayType, DateType
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import ArrayType, StringType, MapType, IntegerType

In [ ]:
review_schema = StructType([
    StructField("index", IntegerType(), True),
    StructField("app_id", LongType(), True),
    StructField("app_name", StringType(), True),
    StructField("review_id", LongType(), True),
    StructField("language", StringType(), True),
    StructField("review", StringType(), True),
    StructField("timestamp_created", IntegerType(), True),
    StructField("timestamp_updated", IntegerType(), True),
    StructField("recommended", BooleanType(), True),
    StructField("votes_helpful", IntegerType(), True),
    StructField("votes_funny", IntegerType(), True),
    StructField("weighted_vote_score", DoubleType(), True),
    StructField("comment_count", IntegerType(), True),
    StructField("steam_purchase", BooleanType(), True),
    StructField("received_for_free", BooleanType(), True),
    StructField("written_during_early_access", BooleanType(), True),
    StructField("author_steamid", LongType(), True),
    StructField("author_num_games_owned", IntegerType(), True),
    StructField("author_num_reviews", IntegerType(), True),
    StructField("author_playtime_forever", DoubleType(), True),
    StructField("author_playtime_last_two_weeks", DoubleType(), True),
    StructField("author_playtime_at_review", DoubleType(), True),
    StructField("author_last_played", DoubleType(), True)
])

In [ ]:
game_schema = StructType([
    StructField("AppID", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("release_date", DateType(), True),
    StructField("required_age", IntegerType(), True),
    StructField("price", FloatType(), True),
    StructField("dlc_count", IntegerType(), True),
    StructField("detailed_description", StringType(), True),
    StructField("about_the_game", StringType(), True),
    StructField("short_description", StringType(), True),
    StructField("reviews", StringType(), True),
    StructField("header_image", StringType(), True),
    StructField("website", StringType(), True),
    StructField("support_url", StringType(), True),
    StructField("support_email", StringType(), True),
    StructField("windows", BooleanType(), True),
    StructField("mac", BooleanType(), True),
    StructField("linux", BooleanType(), True),
    StructField("metacritic_score", IntegerType(), True),
    StructField("metacritic_url", StringType(), True),
    StructField("achievements", IntegerType(), True),
    StructField("recommendations", IntegerType(), True),
    StructField("notes", StringType(), True),
    StructField("supported_languages", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("full_audio_languages", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("packages", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("developers", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("publishers", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("categories", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("genres", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("screenshots", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("movies", ArrayType(StringType()), True),  # Parsed from JSON
    StructField("user_score", FloatType(), True),
    StructField("score_rank", IntegerType(), True),
    StructField("positive", IntegerType(), True),
    StructField("negative", IntegerType(), True),
    StructField("estimated_owners", StringType(), True),
    StructField("average_playtime_forever", IntegerType(), True),
    StructField("average_playtime_2weeks", IntegerType(), True),
    StructField("median_playtime_forever", IntegerType(), True),
    StructField("median_playtime_2weeks", IntegerType(), True),
    StructField("peak_ccu", IntegerType(), True),
    StructField("tags", MapType(StringType(), IntegerType()), True),  # Parsed from JSON
    StructField("pct_pos_total", IntegerType(), True),
    StructField("num_reviews_total", IntegerType(), True),
    StructField("pct_pos_recent", IntegerType(), True),
    StructField("num_reviews_recent", IntegerType(), True)
])

In [ ]:

games = spark\
    .read\
    .option("escapeQuotes", "true")\
    .csv('data/steam_games_flattened.csv', header=True, schema=game_schema, quote='"', escape='"')

game_reviews = spark\
    .read\
    .option("escapeQuotes", "true")\
    .csv('data/steam_game_reviews_flattened.csv', header=True, schema=game_schema, quote='"', escape='"')

In [ ]:
array_schema = ArrayType(StringType())
map_schema = MapType(StringType(), IntegerType())

# Parse the JSON fields
games = games.withColumn("supported_languages", from_json(col("supported_languages"), array_schema)) \
       .withColumn("full_audio_languages", from_json(col("full_audio_languages"), array_schema)) \
       .withColumn("packages", from_json(col("packages"), array_schema)) \
       .withColumn("developers", from_json(col("developers"), array_schema)) \
       .withColumn("publishers", from_json(col("publishers"), array_schema)) \
       .withColumn("categories", from_json(col("categories"), array_schema)) \
       .withColumn("genres", from_json(col("genres"), array_schema)) \
       .withColumn("screenshots", from_json(col("screenshots"), array_schema)) \
       .withColumn("movies", from_json(col("movies"), array_schema)) \
       .withColumn("tags", from_json(col("tags"), map_schema))


In [ ]:
games\
    .write\
    .parquet('data/steam_games_parquet')

In [ ]:
game_reviews_sample = game_reviews.sample(False, 100000/(21 * 10 ** 6), seed=1234)
game_reviews_sample = game_reviews_sample.persist()

In [ ]:
game_reviews.write.parquet("file:////work/ds/steam_reviews_parquet")
games.write.parquet("file:////work/ds/steam_games_parquet")
game_reviews_sample.write.parquet("file:////work/ds/steam_reviews_sample_parquet")